## 정리

대표적으로 유형이 다른 역 5개를 뽑아서 Modeling 진행

선행연구는 코로나를 반영 안하고 LSTM을 이용해서 2023년 1월 ~ 3월 : train / 2023년 4월 : test

코로나의 경우 코로나 정책에서 따라 change point가 생김

따라서, 본 프로젝트에서는 change point를 찾아내고 그 원인을 분석하고 이를 바탕으로 modeling 진행하기

train data : 2019년 ~ 2022년 / test : 2023년

### Step1. Import Libraries

In [1]:
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
from lib import grid_search as gs
from lib import get_data_per_station as sta
from lib import change_point
from lib import sarimax as sa
warnings.filterwarnings('ignore')

### Step2. Load Datasets and Data Processing
Data processing 과정 마무리 후 삭제 하였음. 

데이터 merge 작업과 추가 필요성이 있는 변수들을 추가하는 과정을 거짐.

그 결과가 merged_data.csv

holiday: 설, 추석 연휴 / social_distance_change : 1이면 단계를 올린 시점 -1이면 단계를 내린 시점

In [2]:
merged_df = pd.read_csv("data/processed_data.csv")

In [3]:
def mape(actual, forecast):
    actual, forecast = np.array(actual), np.array(forecast)
    return np.mean(np.abs((actual - forecast) / actual)) * 100

### Step3. EDA

In [ ]:
# stations = merged_df['역명'].unique()
# num_people = [0]*len(stations)
# for i in range(len(stations)):
#     tmp = sta.get_data(merged_df, stations[i])
#     num_people[i] = sum(tmp['승차총승객수']) + sum(tmp['하차총승객수'])
# data = {'stations': stations, 'num_people': num_people}
# tmp = pd.DataFrame(data).sort_values(by='num_people', ascending=False).head(15)
# top_stations = np.array(tmp['stations'])
# print(top_stations)

15개의 역에 대해 그래프를 그린 코드

In [4]:
days = 1826
# for k in range(15):
#     df_station = sta.get_data(merged_df, top_stations[k])
#     tmp = sta.get_tidy_data(df_station)
#     print(top_stations[k])
#     plt.plot(pd.Series(np.array(tmp['하차총승객수'] + tmp['승차총승객수']), index=pd.date_range('2019-01-01', periods=days, freq='D')))
#     plt.show()

In [5]:
station_selected = ['잠실(송파구청)', '강남', '고속터미널', '서울역', '여의도']
names = ['Jamsil Station', 'Gangnam Station', 'Express Bus Terminal', 'Seoul Station', 'Yeouido']

In [12]:
# for k in range(5):
#     df_station = sta.get_data(merged_df, station_selected[k])
#     tmp = sta.get_tidy_data(df_station)
#     plt.plot(pd.Series(np.array(tmp['하차총승객수'] + tmp['승차총승객수']), index=pd.date_range('2019-01-01', periods=days, freq='D')))
#     plt.title(names[k])
#     plt.show()

#### Seasonal Decomposition

In [ ]:
df_station = sta.get_data(merged_df, '잠실(송파구청)')
tmp = sta.get_tidy_data(df_station)
plt.plot(pd.Series(np.array(tmp['하차총승객수'] + tmp['승차총승객수'])[days-365:days], index=pd.date_range('2023-01-01', periods=365, freq='D')))
plt.title("Jamsil Station, 2023")
plt.show()

In [ ]:
ts = pd.Series(np.array(tmp['하차총승객수'] + tmp['승차총승객수'])[days-365:days], index=pd.date_range('2023-01-01', periods=365, freq='D'))
stl = STL(ts); res = stl.fit()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8))
ax1.plot(ts)
ax1.set_title('Original Time Series')
ax2.plot(res.trend)
ax2.set_title('Trend Component')
ax3.plot(res.seasonal)
ax3.set_title('Seasonal Component')
ax4.plot(res.resid)
ax4.set_title('Residual Component')

plt.tight_layout()
plt.show()

#### Outliers

#### 1) 설, 추석 연휴

In [ ]:
counts = tmp[['사용일자', '승차총승객수', '하차총승객수', 'holiday']].copy()
counts['사용일자'] = pd.to_datetime(counts['사용일자'], format='%Y%m%d')
counts.set_index('사용일자', inplace=True)
signal = (counts['승차총승객수'] + counts['하차총승객수']).values
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(counts.index, signal)
ax.set_title('Lunar New Year and Chuseok for Jamsil Station')
plt.xticks(rotation=45)
plt.plot(pd.Series(np.array(counts['holiday']*50000), index=pd.date_range('2019-01-01', periods=days, freq='D')))
plt.show()

#### 2) 잠실 : 4월 첫째주 주말, 크리스마스, and 어린이날

In [ ]:
index = [95, 96, 358, 1187, 1188, 1454, 1551, 1552, 1819, 124, 1220, 1585] #20, 21년 제외
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(counts.index, signal)
for cp in index:
   ax.axvline(counts.index[cp], color='red', linestyle='--')
ax.set_title('Cherry blossom, christmas and children\'s day for Jamsil Station except 20, 21')
plt.xticks(rotation=45)
plt.show()

### Step4. Analysis

#### 잠실 송파구청

In [7]:
df_station = sta.get_data(merged_df, '잠실(송파구청)')
df = sta.get_tidy_data(df_station)

#### 1) change point detection

In [ ]:
change_point.detection(df, 1.5, ' Jamsil')

In [8]:
index = [95, 96, 358, 1187, 1188, 1454, 1551, 1552, 1819, 124, 1220, 1585]  # 코로나 시절의 20, 21년도 제외 4월 첫주 휴일, 크리스마스 및 어린이날
peak = [0]*days
for i in range(days):
    if i in index: peak[i] = 1
df['peak_jamsil'] = peak
df['total'] = df['승차총승객수'] + df['하차총승객수']

In [9]:
df.drop(columns=['승차총승객수', '하차총승객수', 'is_semester', 'is_offline', 'offline_class', 'day_off'], inplace=True)

In [14]:
# df.to_csv("data/df_jamsil.csv", index=False)

#### 2) Fitting

In [10]:
df = pd.read_csv("data/df_jamsil.csv")
station_train = df[0:1461]; station_valid = df[1461:]

train_data = pd.Series(np.array(station_train['total']), index=pd.date_range('2019-01-01', periods=1461, freq='D'))
test_data = pd.Series(np.array(station_valid['total']), index=pd.date_range('2023-01-01', periods=365, freq='D'))

exog_list = ['social_distance_change', 'peak_jamsil', 'holiday']
exog_variables = sta.get_exog_variables(station_train, station_valid, exog_list)

In [ ]:
best_parameter = gs.find_best_parameter(train_data, test_data, exog_variables)
best_parameter

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 7, exog_variables[0], exog_variables[1], "Jamsil Station")

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 14, exog_variables[0], exog_variables[1], "Jamsil Station")

#### 3) Auto-Arima

In [ ]:
ar_model = auto_arima(train_data, seasonal=True, m=7, stepwise=False)
print(ar_model)
fitted = ar_model.fit(train_data).predict_in_sample()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

#### 4) Overfitting Test

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(2, 1, 1), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 0, 2, 7))
print(model.fit().aic)

### 강남역

In [15]:
df_station = sta.get_data(merged_df, '강남')
df = sta.get_tidy_data(df_station)

#### 1) Change Point

In [ ]:
change_point.detection(df, 1.5, ' Gangnam')

In [17]:
index = [124, 1220, 1585] # 코로나 없던 어린이날에 change point
peak = [0]*days
for i in range(days):
    if i in index: peak[i] = 1
df['peak_gangnam'] = peak
df['total'] = df['승차총승객수'] + df['하차총승객수']
df.drop(columns=['승차총승객수', '하차총승객수', 'is_semester', 'is_offline', 'offline_class', 'day_off'], inplace=True)

In [18]:
# df.to_csv("data/df_gangnam.csv", index=False)

### 2) fitting

In [22]:
df = pd.read_csv("data/df_gangnam.csv")
station_train = df[0:1461]; station_valid = df[1461:]

train_data = pd.Series(np.array(station_train['total']), index=pd.date_range('2019-01-01', periods=1461, freq='D'))
test_data = pd.Series(np.array(station_valid['total']), index=pd.date_range('2023-01-01', periods=365, freq='D'))

exog_list = ['social_distance_change', 'holiday', 'peak_gangnam']
exog_variables = sta.get_exog_variables(station_train, station_valid, exog_list)

In [ ]:
best_parameter = gs.find_best_parameter(train_data, test_data, exog_variables)
best_parameter

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 7, exog_variables[0], exog_variables[1], "Gangnam Station")

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 14, exog_variables[0], exog_variables[1], "Gangnam Station")

#### 3) Auto_arima

In [ ]:
ar_model = auto_arima(train_data, max_P=4, max_Q=4, seasonal=True, m=7, stepwise=False)
print(ar_model)
fitted = ar_model.fit(train_data).predict_in_sample()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

#### 4) Overfitting Test

In [ ]:
model = SARIMAX(train_data, order=(0, 1, 1), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(0, 1, 2), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(0, 1, 1), seasonal_order=(3, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(0, 1, 1), seasonal_order=(2, 0, 2, 7))
print(model.fit().aic)

In [ ]:
model = SARIMAX(train_data, order=(0, 1, 1), seasonal_order=(2, 0, 1, 7))
fitted = model.fit().predict()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

### 여의도

In [31]:
df_station = sta.get_data(merged_df, '여의도')
df = sta.get_tidy_data(df_station)

#### 1) Change Point

In [ ]:
change_point.detection(df, 1.5, ' Yeouido')

In [32]:
index_1 = [276, 1375, 1740] #불꽃 축제
peak_1 = [0]*days
peak_2 = [0]*days
for i in range(days):
    if i in index_1: peak_1[i] = 1
    #if i in index_2: peak_2[i] = 1
df['peak_yeouido'] = peak_1
#df['peak_yeouido_2'] = peak_2
df['total'] = df['승차총승객수'] + df['하차총승객수']
df.drop(columns=['승차총승객수', '하차총승객수', 'is_semester', 'is_offline', 'offline_class', 'day_off'], inplace=True)

In [33]:
df.to_csv("data/df_yeouido.csv", index=False)

### 2) fitting

In [27]:
df = pd.read_csv("data/df_yeouido.csv")
station_train = df[0:1461]; station_valid = df[1461:]

train_data = pd.Series(np.array(station_train['total']), index=pd.date_range('2019-01-01', periods=1461, freq='D'))
test_data = pd.Series(np.array(station_valid['total']), index=pd.date_range('2023-01-01', periods=365, freq='D'))

exog_list = ['social_distance_change', 'holiday', 'peak_yeouido']
exog_variables = sta.get_exog_variables(station_train, station_valid, exog_list)

In [ ]:
best_parameter = gs.find_best_parameter(train_data, test_data, exog_variables)
best_parameter

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 7, exog_variables[0], exog_variables[1], "Yeouido Station")

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 14, exog_variables[0], exog_variables[1], "Yeouido Station")

#### 3) Auto-Arima

In [ ]:
ar_model = auto_arima(train_data, seasonal=True, m=7, stepwise=False)
print(ar_model)
fitted = ar_model.fit(train_data).predict_in_sample()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

#### 4) Overfitting Test

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(2, 1, 2), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 2, 7))
print(model.fit().aic)

### 고속터미널

In [43]:
df_station = sta.get_data(merged_df, '고속터미널')
df = sta.get_tidy_data(df_station)

#### 1) Change point

In [ ]:
change_point.detection(df, 1.5, ' Express Bus Treminal')

In [44]:
index_1 = [124, 1220, 1585] #20 21 제외 어린이날
index_2 = [35, 255, 389, 639, 773, 994, 1127, 1348, 1483, 1732] #설 당일, 추석 당일
index_3 = [34, 36, 254, 256, 388, 390, 638, 640, 772, 774, 993, 995, 1126, 1128, 1347, 1349, 1482, 1484, 1731, 1733] #설, 추석 전날, 다음 날
index_4 = [0, 364, 365, 730, 731, 1095, 1096, 1460, 1461, 1825] #새해 첫날, 마지막 날
peak_1 = [0]*days
peak_2 = [0]*days
peak_3 = [0]*days
peak_4 = [0]*days
for i in range(days):
    if i in index_1: peak_1[i] = 1
    if i in index_2: peak_2[i] = 1
    if i in index_3: peak_3[i] = 1
    if i in index_4: peak_4[i] = 1
df['peak_express_1'] = peak_1
df['peak_express_2'] = peak_2
df['peak_express_3'] = peak_3
df['peak_express_4'] = peak_4
df['total'] = df['승차총승객수'] + df['하차총승객수']
df.drop(columns=['승차총승객수', '하차총승객수', 'is_semester', 'is_offline', 'offline_class', 'day_off'], inplace=True)

In [45]:
# df.to_csv("data/df_express_terminal.csv", index=False)

#### 2) fitting

In [30]:
df = pd.read_csv("data/df_express_terminal.csv")
station_train = df[0:1461]; station_valid = df[1461:]

train_data = pd.Series(np.array(station_train['total']), index=pd.date_range('2019-01-01', periods=1461, freq='D'))
test_data = pd.Series(np.array(station_valid['total']), index=pd.date_range('2023-01-01', periods=365, freq='D'))

exog_list = ['social_distance_change', 'peak_express_1', 'peak_express_2', 'peak_express_3', 'peak_express_4']
exog_variables = sta.get_exog_variables(station_train, station_valid, exog_list)

In [ ]:
best_parameter = gs.find_best_parameter(train_data, test_data, exog_variables)
best_parameter

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 7, exog_variables[0], exog_variables[1], "Express Bus Terminal Station")

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 14, exog_variables[0], exog_variables[1], "Express Bus Terminal Station")

#### 3) Auto Arima

In [ ]:
ar_model = auto_arima(train_data, seasonal=True, m=7, stepwise=False)
print(ar_model)
fitted = ar_model.fit(train_data).predict_in_sample()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

#### 4) Overfitting Test

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(2, 1, 3), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 4), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(1, 0, 2, 7))
print(model.fit().aic)

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(1, 0, 1, 7))
fitted = model.fit().predict()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

### 서울역

In [39]:
df_station = sta.get_data(merged_df, '서울역')
df = sta.get_tidy_data(df_station)

#### 1) Change point

In [ ]:
change_point.detection(df, 1.5, ' Seoul Station')

In [41]:
index_1 = [124, 1220, 1585] #20 21 제외 어린이날
index_2 = [35, 255, 389, 639, 773, 994, 1127, 1348, 1483, 1732] #설 당일, 추석 당일
index_3 = [34, 36, 254, 256, 388, 390, 638, 640, 772, 774, 993, 995, 1126, 1128, 1347, 1349, 1482, 1484, 1731, 1733] #설, 추석 전날, 다음 날 
peak_1 = [0]*days
peak_2 = [0]*days
peak_3 = [0]*days
for i in range(days):
    if i in index_1: peak_1[i] = 1
    if i in index_2: peak_2[i] = 1
    if i in index_3: peak_3[i] = 1
df['peak_seoul_1'] = peak_1
df['peak_seoul_2'] = peak_2
df['peak_seoul_3'] = peak_3
df['total'] = df['승차총승객수'] + df['하차총승객수']
df.drop(columns=['승차총승객수', '하차총승객수', 'is_semester', 'is_offline', 'offline_class', 'day_off'], inplace=True)

In [42]:
# df.to_csv("data/df_seoul.csv", index=False)

#### 2) fitting

In [35]:
df= pd.read_csv("data/df_seoul.csv")
station_train = df[0:1461]; station_valid = df[1461:]

train_data = pd.Series(np.array(station_train['total']), index=pd.date_range('2019-01-01', periods=1461, freq='D'))
test_data = pd.Series(np.array(station_valid['total']), index=pd.date_range('2023-01-01', periods=365, freq='D'))

exog_list = ['social_distance_change', 'peak_seoul_1','peak_seoul_2', 'peak_seoul_3']
exog_variables = sta.get_exog_variables(station_train, station_valid, exog_list)

In [ ]:
best_parameter = gs.find_best_parameter(train_data, test_data, exog_variables)
best_parameter

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 7, exog_variables[0], exog_variables[1], "Seoul Station")

In [ ]:
sa.sarimax_result(train_data, test_data, best_parameter[0:3], best_parameter[3:6], 14, exog_variables[0], exog_variables[1], "Seoul Station")

In [ ]:
ar_model = auto_arima(train_data, seasonal=True, m=7, stepwise=False)
print(ar_model)
fitted = ar_model.fit(train_data).predict_in_sample()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))

#### 4) Overfitting Test

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 3), seasonal_order=(1, 0, 1, 7)) #fitting error
print(model.fit().aic)
model = SARIMAX(train_data, order=(2, 1, 2), seasonal_order=(1, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(2, 0, 1, 7))
print(model.fit().aic)
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 2, 7))
print(model.fit().aic)

In [ ]:
model = SARIMAX(train_data, order=(1, 1, 2), seasonal_order=(1, 0, 1, 7))
fitted = model.fit().predict()
tr_rmse = np.sqrt(np.mean((fitted - train_data)**2))
print("train RMSE :", tr_rmse)
print("train MAPE :", mape(train_data, fitted))
predictions = ar_model.predict(365)
rmse = np.sqrt(np.mean((test_data - predictions)**2))
print("test RMSE :", rmse)
print("test MAPE :", mape(test_data, predictions))